# テンソル
テンソルは、配列や行列に非常によく似た特殊なデータ構造です。  
PyTorch では、テンソルを使用してモデルの入力と出力、およびモデルのパラメータをエンコードします。

テンソルはNumPy のndarrayに似ていますが、テンソルは GPU やその他のハードウェア アクセラレータで実行できる点が異なります。  
実際、テンソルと NumPy 配列は多くの場合、同じ基礎メモリを共有できるため、データをコピーする必要がありません ( 「NumPy とのブリッジ」を参照)。  
テンソルは自動微分にも最適化されています (これについては、 Autograd のセクションで後ほど詳しく説明します )。  
ndarray に精通していれば、Tensor API もすぐに使いこなせるでしょう。  
そうでない場合は、このまま読み進めてください。

# テンソルの初期化
テンソルはさまざまな方法で初期化できます。次の例を見てみましょう。

## データから直接

テンソルはデータから直接作成できます。データ型は自動的に推測されます。

In [ ]:
import torch
import numpy as np

data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.6045, 0.1330],
        [0.7897, 0.6708]]) 



## NumPy配列から

テンソルは NumPy 配列から作成できます (逆も同様です - Bridge with NumPy を参照)。

In [7]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

## 別のテンソルから:

新しいテンソルは、明示的にオーバーライドされない限り、引数テンソルのプロパティ (形状、データ型) を保持します。

In [8]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.8592, 0.0056],
        [0.2401, 0.4620]]) 



## ランダム値または定数値の場合:

shapeテンソルの次元のタプルです。以下の関数では、出力テンソルの次元を決定します。

In [3]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.1346, 0.4219, 0.0915],
        [0.7220, 0.8685, 0.0719]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


# テンソルの属性
テンソル属性は、テンソルの形状、データ型、およびテンソルが保存されているデバイスを記述します。

In [10]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


# テンソルの演算
ここでは、算術、線形代数、行列操作 (転置、インデックス作成、スライス)、サンプリングなど、1200 を超えるテンソル演算が包括的に説明されています。

これらの各操作は、CPU と、CUDA、MPS、MTIA、XPU などのアクセラレータで実行できます 。  
Colab を使用している場合は、[ランタイム] > [ランタイム タイプの変更] > [GPU] に移動してアクセラレータを割り当てます。

デフォルトでは、テンソルは CPU 上に作成されます。  
 .toアクセラレータの可用性を確認した後、メソッドを使用してテンソルをアクセラレータに明示的に移動する必要があります。  
 デバイス間で大きなテンソルをコピーすると、時間とメモリの面でコストがかかる可能性があることに注意してください。

In [12]:
# We move our tensor to the current accelerator if available
if torch.cuda.is_available():
    tensor = tensor.to(torch.device('cuda'))
else:
    tensor = tensor.to(torch.device('cpu'))

リストにある操作をいくつか試してみてください。NumPy API に慣れている方なら、Tensor API も簡単に使えるでしょう。

## 標準的な numpy のようなインデックスとスライス:

In [ ]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


## テンソルの結合 
をtorch.cat使用すると、指定された次元に沿ってテンソルのシーケンスを連結できます。　　
とは微妙に異なる別のテンソル結合演算子であるtorch.stacktorch.catも参照してください。

In [3]:
tensor = torch.ones(4, 4)
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


# 算術演算

In [5]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

# 単一要素テンソル
単一要素テンソルがある場合、たとえばテンソルのすべての値を 1 つの値に集約すると、次のようにして Python 数値に変換できますitem()。

In [6]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

16.0 <class 'float'>


## インプレース 
演算 結果をオペランドに格納する演算は、インプレースと呼ばれます。  
これらは_接尾辞で示されます。たとえばx.copy_(y)、x.t_()、 は を変更しますx。

In [13]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[0.8021, 0.7149, 0.9758, 0.9357],
        [0.3192, 0.0297, 0.5533, 0.5603],
        [0.4547, 0.5924, 0.6394, 0.3965]]) 

tensor([[5.8021, 5.7149, 5.9758, 5.9357],
        [5.3192, 5.0297, 5.5533, 5.5603],
        [5.4547, 5.5924, 5.6394, 5.3965]])


インプレース演算はメモリをいくらか節約しますが、履歴がすぐに失われるため導関数を計算するときに問題が発生する可能性があります。
したがって、インプレース演算の使用は推奨されません。

# NumPy を使用したブリッジ
CPU 上のテンソルと NumPy 配列は、基礎となるメモリの場所を共有できるため、一方を変更するともう一方も変更されます。

## テンソルからNumPy配列へ

In [14]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


テンソルの変更は NumPy 配列に反映されます。

In [15]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


## NumPy配列からTensorへ

In [16]:
n = np.ones(5)
t = torch.from_numpy(n)

NumPy 配列の変更はテンソルに反映されます。

In [17]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
